## Basics

#### Start your client

In [1]:
import dask
from dask.distributed import Client, progress

try:
    client.close()
    client = Client(threads_per_worker=1, n_workers=5, memory_limit='2GB')
except:
    client = Client(threads_per_worker=1, n_workers=5, memory_limit='2GB')

client

Client Scheduler: tcp://127.0.0.1:45094 Dashboard: http://127.0.0.1:8787/status,Cluster Workers: 5 Cores: 5 Memory: 10.00 GB


## Sample Runs

In [2]:
%pylab inline

import pandas as pd
from dask import delayed
from tqdm.notebook import tqdm

Populating the interactive namespace from numpy and matplotlib


In [8]:
n = 1000

In [9]:
def simulate_dataset(i):
    
    g = 10
    k = 100000

    categories = np.random.choice(a = np.arange(g), size= k)
    values = [(j+1)*np.random.random(k//g) for j in range(g) ]
    values = np.concatenate(values)
    
    data = pd.DataFrame({'category':categories, 'values':values})
    data_out = data.groupby('category').apply(lambda df: [df.loc[:,'values'].mean(),
                                                          df.loc[:,'values'].std()])
    
    return(data_out)

In [10]:
%%timeit

results = {}

for i in range(n):
    
    results[i] = simulate_dataset(i)

14.1 s ± 94.8 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)


In [12]:
%%timeit

results_interim = {}
for i in range(n):
    results_interim[i] = delayed(simulate_dataset)(i)

results_scheduled = delayed(list)(results_interim)
results = results_scheduled.compute()

5.72 s ± 60.7 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)
